# Detecting Face Masks in Videos

## Import Modules

In [ ]:
# Use Tensorflow 2+
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os

## Load Models

In [ ]:
facenet = cv2.dnn.readNet('models/deploy.prototxt', 'models/res10_300x300_ssd_iter_140000.caffemodel')
model = load_model('models/mask_detector.model')

## Load the Video

In [ ]:
video = cv2.VideoCapture('imgs/01.mp4')
ret, img = video.read()

## Preparation for Face Detection of Video

In [ ]:
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
out = cv2.VideoWriter('output.mp4', fourcc, video.get(cv2.CAP_PROP_FPS), (img.shape[1], img.shape[0]))

## Detecting Face Masks in Video

In [ ]:
while video.isOpened():
    ret, img = video.read()
    if not ret:
        break
    
    h, w = img.shape[:2]
    
    blob = cv2.dnn.blobFromImage(img, 1.0, (300, 300), (104.0, 177.0, 123.0))
    facenet.setInput(blob)
    dets = facenet.forward()
    
    result_img = img.copy()
    
    for i in range(dets.shape[2]):
        confidence = dets[0, 0, i, 2]
        if confidence > 0.5:
            
            x1 = int(dets[0, 0, i, 3] * w)
            y1 = int(dets[0, 0, i, 4] * h)
            x2 = int(dets[0, 0, i, 5] * w)
            y2 = int(dets[0, 0, i, 6] * h)
            
            face = img[y1:y2, x1:x2]
            
            face_mask = cv2.resize(face, (224, 224))
            face_mask = cv2.cvtColor(face_mask, cv2.COLOR_BGR2RGB)
            face_mask = preprocess_input(face_mask)
            face_mask = np.expand_dims(face_mask, axis=0)
            
            mask, nomask = model.predict(face_mask).squeeze()
            
            if mask > nomask:
                color = (0, 255, 0)
                label = 'Mask %d%%' % (mask * 100)
            else:
                color = (0, 0, 255)
                label = 'No Mask %d%%' % (nomask * 100)
            
            cv2.rectangle(result_img, (x1, y1), (x2, y2), color, 2)
            cv2.putText(result_img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            
    out.write(result_img)
    cv2.imshow('result', result_img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
out.release()
video.release()    
cv2.destroyAllWindows()